In [1]:
import pandas as pd
import numpy as np

In [2]:
dataset=pd.read_csv('CO2_emission.csv')
dataset

,Model_Year,Make,Model,Vehicle_Class,Engine_Size,Cylinders,Transmission,Fuel_Consumption_in_City(L/100 km),Fuel_Consumption_in_City_Hwy(L/100 km),Fuel_Consumption_comb(L/100km),CO2_Emissions,Smog_Level
0,2021,Acura,ILX,Compact,2.4,4,AM8,9.9,7.0,8.6,199,3
1,2021,Acura,NSX,Two-seater,3.5,6,AM9,11.1,10.8,11.0,256,3
2,2021,Acura,RDX SH-AWD,SUV: Small,2.0,4,AS10,11.0,8.6,9.9,232,6
3,2021,Acura,RDX SH-AWD A-SPEC,SUV: Small,2.0,4,AS10,11.3,9.1,10.3,242,6
4,2021,Acura,TLX SH-AWD,Compact,2.0,4,AS10,11.2,8.0,9.8,230,7
...,...,...,...,...,...,...,...,...,...,...,...,...
930,2021,Volvo,XC40 T5 AWD,SUV: Small,2.0,4,AS8,10.7,7.7,9.4,219,5
931,2021,Volvo,XC60 T5 AWD,SUV: Small,2.0,4,AS8,11.1,8.3,9.9,230,5
932,2021,Volvo,XC60 T6 AWD,SUV: Small,2.0,4,AS8,11.7,8.6,10.3,240,7
933,2021,Volvo,XC90 T5 AWD,SUV: Standard,2.0,4,AS8,11.5,8.4,10.1,236,5


In [3]:
#dataset=pd.get_dummies(dataset,drop_first=True)
#dataset

# Getting basic info about dataframe:

In [4]:
dataset.shape

(935, 12)

In [5]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 935 entries, 0 to 934
Data columns (total 12 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   Model_Year                              935 non-null    int64  
 1   Make                                    935 non-null    object 
 2   Model                                   935 non-null    object 
 3   Vehicle_Class                           935 non-null    object 
 4   Engine_Size                             935 non-null    float64
 5   Cylinders                               935 non-null    int64  
 6   Transmission                            935 non-null    object 
 7   Fuel_Consumption_in_City(L/100 km)      935 non-null    float64
 8   Fuel_Consumption_in_City_Hwy(L/100 km)  935 non-null    float64
 9   Fuel_Consumption_comb(L/100km)          935 non-null    float64
 10  CO2_Emissions                           935 non-null    int64 

In [6]:
dataset.describe()

,Model_Year,Engine_Size,Cylinders,Fuel_Consumption_in_City(L/100 km),Fuel_Consumption_in_City_Hwy(L/100 km),Fuel_Consumption_comb(L/100km),CO2_Emissions,Smog_Level
count,935.0,935.000000,935.000000,935.000000,935.000000,935.000000,935.000000,935.000000
mean,2021.0,3.214866,5.716578,12.498610,9.306203,11.060214,258.529412,4.726203
std,0.0,1.388513,1.977359,3.487271,2.215819,2.867028,64.442768,1.712127
min,2021.0,1.000000,3.000000,4.000000,3.900000,4.000000,94.000000,1.000000
25%,2021.0,2.000000,4.000000,10.100000,7.700000,9.100000,213.000000,3.000000
50%,2021.0,3.000000,6.000000,12.000000,9.000000,10.700000,255.000000,5.000000
75%,2021.0,4.000000,8.000000,14.800000,10.800000,13.100000,303.500000,6.000000
max,2021.0,8.000000,16.000000,30.300000,20.900000,26.100000,608.000000,7.000000


# Cleaning the dataset:

In [7]:
dataset.isna().sum()

Model_Year                                0
Make                                      0
Model                                     0
Vehicle_Class                             0
Engine_Size                               0
Cylinders                                 0
Transmission                              0
Fuel_Consumption_in_City(L/100 km)        0
Fuel_Consumption_in_City_Hwy(L/100 km)    0
Fuel_Consumption_comb(L/100km)            0
CO2_Emissions                             0
Smog_Level                                0
dtype: int64

In [8]:
dataset[dataset.duplicated()]

,Model_Year,Make,Model,Vehicle_Class,Engine_Size,Cylinders,Transmission,Fuel_Consumption_in_City(L/100 km),Fuel_Consumption_in_City_Hwy(L/100 km),Fuel_Consumption_comb(L/100km),CO2_Emissions,Smog_Level


In [9]:
dataset.columns

Index(['Model_Year', 'Make', 'Model', 'Vehicle_Class', 'Engine_Size',
       'Cylinders', 'Transmission', 'Fuel_Consumption_in_City(L/100 km)',
       'Fuel_Consumption_in_City_Hwy(L/100 km)',
       'Fuel_Consumption_comb(L/100km)', 'CO2_Emissions', 'Smog_Level'],
      dtype='object')

# Univariate Analysis

In [10]:
qual=[]
quan=[]

for columnName in dataset.columns:
    if dataset[columnName].dtype=='object':
        qual.append(columnName)
    else:
        quan.append(columnName)

In [11]:
qual

['Make', 'Model', 'Vehicle_Class', 'Transmission']

In [12]:
quan

['Model_Year',
 'Engine_Size',
 'Cylinders',
 'Fuel_Consumption_in_City(L/100 km)',
 'Fuel_Consumption_in_City_Hwy(L/100 km)',
 'Fuel_Consumption_comb(L/100km)',
 'CO2_Emissions',
 'Smog_Level']

In [13]:
#pd.options.mode.copy_on_write=True

In [14]:
Univariate=pd.DataFrame(columns=[quan],
                        index=['Mean','Median','Mode','Min','Q1:25%','Q2:50%','Q3:75%','99%','Q4:100%','IQR','Lesser_IQR','Greater_IQR'])
    
for columnName in quan:
    print(columnName)

Model_Year
Engine_Size
Cylinders
Fuel_Consumption_in_City(L/100 km)
Fuel_Consumption_in_City_Hwy(L/100 km)
Fuel_Consumption_comb(L/100km)
CO2_Emissions
Smog_Level


In [15]:
Univariate=pd.DataFrame(columns=[quan],
                        index=['Mean','Median','Mode','Min','Q1:25%','Q2:50%','Q3:75%','99%','Q4:100%','IQR','Lesser_IQR','Greater_IQR'])
    
for columnName in quan:
    Univariate.loc['Mean', columnName]=dataset[columnName].mean()
    Univariate.loc['Median',columnName]=dataset[columnName].median()
    Univariate.loc['Mode',columnName]=dataset[columnName].mode()[0]
    Univariate.loc['Min',columnName]=dataset[columnName].min()
    Univariate.loc['Q1:25%',columnName]=np.percentile(dataset[columnName],25)
    Univariate.loc['Q2:50%',columnName]=np.percentile(dataset[columnName],50)
    Univariate.loc['Q3:75%',columnName]=np.percentile(dataset[columnName],75)
    Univariate.loc['99%',columnName]=np.percentile(dataset[columnName],99)
    Univariate.loc['Q4:100%',columnName]=dataset[columnName].max()
    Univariate.loc['IQR',columnName]=Univariate.loc['Q3:75%',columnName]-Univariate.loc['Q1:25%',columnName]
    Univariate.loc['Lesser_IQR',columnName]=Univariate.loc['Q1:25%',columnName]-1.5*Univariate.loc['IQR',columnName]
    Univariate.loc['Greater_IQR',columnName]=Univariate.loc['Q3:75%',columnName]+1.5*Univariate.loc['IQR',columnName]

In [16]:
Univariate

,Model_Year,Engine_Size,Cylinders,Fuel_Consumption_in_City(L/100 km),Fuel_Consumption_in_City_Hwy(L/100 km),Fuel_Consumption_comb(L/100km),CO2_Emissions,Smog_Level
Mean,2021.0,3.214866,5.716578,12.49861,9.306203,11.060214,258.529412,4.726203
Median,2021.0,3.0,6.0,12.0,9.0,10.7,255.0,5.0
Mode,2021,2.0,4,10.1,8.7,9.1,214,5
Min,2021,1.0,3,4.0,3.9,4.0,94,1
Q1:25%,2021.0,2.0,4.0,10.1,7.7,9.1,213.0,3.0
Q2:50%,2021.0,3.0,6.0,12.0,9.0,10.7,255.0,5.0
Q3:75%,2021.0,4.0,8.0,14.8,10.8,13.1,303.5,6.0
99%,2021.0,6.6,12.0,21.864,15.9,19.432,408.0,7.0
Q4:100%,2021,8.0,16,30.3,20.9,26.1,608,7
IQR,0.0,2.0,4.0,4.7,3.1,4.0,90.5,3.0


In [19]:
lesser_range=[]
greater_range=[]

for columnName in quan:
    if Univariate[columnName]['Min'] < Univariate[columnName]['Lesser_IQR']:
        lesser_range.append(columnName)
    if Univariate[columnName]['Q4:100%'] > Univariate[columnName]['Greater_IQR']:
        greater_range.append(columnName)

KeyError: 'Min'